

 # <font color='red'>Documents Classification NLP and Naive Bayes</font>

 - Datasets: http://qwone.com/~jason/20Newsgroups/
 - http://scikit-learn.org/stable/modules/naive_bayes.html
 - http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html


## Libraries

In [101]:
## Pip Install

# !pip install pandas
# # !pip install inflection
# # !pip install sqlalchemy
# # !pip install geopy
# # !pip install pycountry-convert
# # !pip install seaborn
# # !pip install plotly
# # !pip install pycountry-convert
# # !pip install melt
# !pip install sklearn
# # !pip install Image
# # !pip install missingno
# # !pip install boruta
# # !pip install xgboost
# # !pip install xlrd
# # !pip install -U notebook-as-pdf
# # !pyppeteer-install
! pip install wordcloud 

  Using cached matplotlib-3.5.1-cp38-cp38-win_amd64.whl (7.2 MB)
  Using cached Pillow-9.0.1-cp38-cp38-win_amd64.whl (3.2 MB)
  Using cached fonttools-4.29.1-py3-none-any.whl (895 kB)
  Using cached kiwisolver-1.3.2-cp38-cp38-win_amd64.whl (52 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)


In [105]:
# Imports
import numpy as np
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

##  Data Acquisition

In [19]:
fetch_20newsgroups

<function sklearn.datasets._twenty_newsgroups.fetch_20newsgroups(*, data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)>

## Data Preparation

In [53]:
#using only 4 categories
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [54]:
# Train
twenty_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42) #shuffle=random

In [55]:
# Classes
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [56]:
# Lenght
len(twenty_train.data)

2257

## Data visualization 

In [62]:
# Atributes Visualization
print("\n".join(twenty_train.data[0].split("\n")[:10]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.



In [63]:
#  Target visualization
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [64]:
# labels - array 
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [65]:
# Records
for t in twenty_train.target[:15]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med
soc.religion.christian
comp.graphics
alt.atheism
alt.atheism
comp.graphics


In [112]:
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate()
plt.imshow(twenty_train , interpolation = 'bilinear')

TypeError: generate() missing 1 required positional argument: 'text'

<Figure size 1440x1440 with 0 Axes>

## Bag of Words

### Tokenizing

In [71]:

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
count_vect.vocabulary_.get(u'algorithm')
X_train_counts.shape

(2257, 35788)

### Term Frequency times Inverse Document Frequency (Tfidf)

In [72]:
tf_transformer = TfidfTransformer(use_idf = False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [74]:
# Mesmo resultado da célula anterior, mas combinando as funções
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

## Machine Learning Modelling

In [77]:
# Multinomial Model
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [82]:
# Forecast _ example
docs_new = ['Darwin fish bumper stickers']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'Darwin fish bumper stickers' => alt.atheism


## Pipeline

In [85]:
# vectorizer => transformer => classifier 
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [86]:
# Fit
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

## Performance Metrics

### Model accuracy

In [90]:

twenty_test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)    

0.8348868175765646

In [91]:

print(metrics.classification_report(twenty_test.target, predicted, target_names = twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



### Confusion Matrix

In [92]:

metrics.confusion_matrix(twenty_test.target, predicted)

array([[192,   2,   6, 119],
       [  2, 347,   4,  36],
       [  2,  11, 322,  61],
       [  2,   2,   1, 393]], dtype=int64)

## Fine Tunning

In [94]:
# Parâmetros para o GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

In [95]:
# GridSearchCV
gs_clf = GridSearchCV(text_clf, parameters, n_jobs = -1)

In [96]:
# Fit
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [97]:
# Teste
twenty_train.target_names[gs_clf.predict(['atheist paraphernalia'])[0]]

'alt.atheism'

In [98]:
# Score
gs_clf.best_score_        

0.9349999999999999

In [99]:
# Parâmetros utilizados
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 2)
